In [151]:
# 일단 아는애들 불러오고

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.font_manager as fm # 한글 폰트 사용

# !pip install missingno # 결측치 분석 전용 라이브러리

  Using cached missingno-0.5.2-py3-none-any.whl.metadata (639 bytes)
Using cached missingno-0.5.2-py3-none-any.whl (8.7 kB)


In [197]:
# 데이터 살펴보기

# csv파일 이니셜로 dataframe명
#df = pd.read_csv('internet_service_churn.csv')

# shape 72274행 11열
#df.shape

# 데이터 기본 정보 총 11개의 행이 모두 숫자형(int64 7개, float32 4개)
#print(df.info())

# 데이터 타입 11행 모두 숫자형 int64, float32
#df.dtypes

# 데이터 타입별 갯수 int64 7개, float32 4개
#df.dtypes.value_counts()

#습관적으로 10개 행 보기
#df.head(10)

# 데이터 컬럼명 확인
#list(isc_df.columns)

# 간단하게 결측치와 이상치 탐색 - 평균, 중위값 등과 차이가 큰 컬럼 일단 의심
# 결측치 3개 확인 reamining_contract, download_avg, upload_avg
# 이상치 의심 subscription_age, bill_avg, reamining_contract, service_failure_count, download_avg, upload_avg
#round(df.describe(), 2)

# target = churn 적정성 확인 1 = 40,050개 0 = 32,224
#isc_df['churn'].value_counts()

# 1과 0의 비율 확인 1 = 55% 0 = 45%
# round(df['churn'].value_counts(normalize=True), 2)

# 타겟컬럼(1이탈, 0유지)과의 상관계수 확인 id, churn 제외
# relate = round(df.corr(numeric_only=True)['churn'].drop(['id', 'churn']).sort_values(ascending=True), 4)
# relate_df = pd.DataFrame({
#     '컬럼명' : relate.index,
#     '상관계수' : relate.values
# })
# display(relate_df)
#
# # 상관계수 시각화
# plt.rcParams['font.family'] = 'Malgun Gothic' # 한글 폰트 표현
# plt.rcParams['axes.unicode_minus'] = False # 마이너스(-) 표현
# plt.figure(figsize=(10, 6))
# plt.barh(relate_df['컬럼명'], relate_df['상관계수'])
# plt.xlabel('churn과의 상관계수')
# plt.title('각 수치형 컬럼과 churn 간의 상관관계')
# plt.grid(axis='x', linestyle='--', alpha=0.7)
# plt.tight_layout()
# plt.show()

# 결측치 확인 - 위 df.describe()에서 발견한 컬럼
# reamining_contract, download_avg, upload_avg

# reamining_contract
# miss_reamining = pd.DataFrame([{
#     '컬럼명': 'reamining_contract',
#     '전체 개수': len(df['reamining_contract']),
#     '결측치 수': df['reamining_contract'].isnull().sum(),
#     '결측 비율 (%)': round(df['reamining_contract'].isnull().mean() * 100, 2)
# }])
#
# # download_avg
# miss_download = pd.DataFrame([{
#     '컬럼명': 'download_avg',
#     '전체 개수': len(df['download_avg']),
#     '결측치 수': df['download_avg'].isnull().sum(),
#     '결측 비율 (%)': round(df['download_avg'].isnull().mean() * 100, 2)
# }])
#
# # upload_avg
# miss_upload = pd.DataFrame([{
#     '컬럼명': 'upload_avg',
#     '전체 개수': len(df['upload_avg']),
#     '결측치 수': df['upload_avg'].isnull().sum(),
#     '결측 비율 (%)': round(df['upload_avg'].isnull().mean() * 100, 2)
# }])
#
# concat으로 프레임 연결
# miss_df = pd.concat([miss_reamining, miss_download, miss_upload], ignore_index=True)
# display(miss_df)

# 1. 결측치 처리 reamining_contract 72,274개 중 21,572개 29.85% 결측
# NaN을 0.0으로 채우고 처리한 컬럼 생성
# df['fill_reamining_contract'] = df['reamining_contract'].fillna(0.0)
# print(df['fill_reamining_contract'].isnull().sum())

# 2. 걸측치 처리 download_avg 72,274개 중 381개 0.53%
# 0.0로 채우고 처리한 컬럼 생성
# df['fill_download_avg'] = df['download_avg'].fillna(0.0)
# print(df['fill_download_avg'].isnull().sum())

# 3. 결측치 처리 upload_avg 72,274개 중 381개 0.53%
# 0.0으로 채우고 처리한 컬럼 생성
# df['fill_upload_avg'] = df['upload_avg'].fillna(0.0)
# print(df['fill_upload_avg'].isnull().sum())

# 이상치 처리
#reamining_contract
# 사분위수 (Q1, Q3) 계산
# Q1 = df['subscription_age'].quantile(0.25) # 데이터의 하위 25%
# Q3 = df['subscription_age'].quantile(0.75) # 데이터의 상위 25%
# # Interquartile Range 계산
# IQR = Q3 - Q1 # 데이터 중 중간 50%가 차지하는 범위
# # 이상치 경계 계산
# low_border = Q1 - 1.5 * IQR # Q1 아래로 1.5배 더 떨어진 값
# up_border = Q3 + 1.5 * IQR #  Q3 위로 1.5배 더 커진 값
#
# # 이상치 탐지 - low_num 보다 작거나 , up_num 보다 크면 이상치
# outliers = df[(df['subscription_age'] < low_border) | (df['subscription_age'] > up_border)]
# # 이상치 갯수와 비율
# outlier_count = outliers['subscription_age'].count()
# total_count = df['subscription_age'].notna().sum()
# outlier_ratio = round((outlier_count / total_count) * 100, 2)
#
# # 하한 이상치 수
# low_count = df[df['subscription_age'] < low_border]['subscription_age'].count()
# # 상한 이상치 수
# up_count = df[df['subscription_age'] > up_border]['subscription_age'].count()
#
# # 결과 요약
# iqr_subscription_age = pd.DataFrame({
#     '통계값': ['Q1 (25%)','Q3 (75%)','IQR','이상치 하한','이상치 상한',
#         '하한 미만 개수','상한 초과 개수','이상치 총 개수','이상치 비율 (%)'],
#     '값': [Q1,Q3,IQR,low_border,up_border,f'{low_count}개',f'{up_count}개',
#         f'{outlier_count}개',f'{outlier_ratio} %']
# })
# display(iqr_subscription_age)

# 이상치 분석 함수
# def get_outlier_summary(df, exclude=[]):
#     """
#     지정한 수치형 컬럼들에 대해 IQR 기반 이상치 통계 요약표 생성
#     """
#     summary_list = [] # 결과값 넣을 빈 리스트
#     # 수치형 컬럼만 가져오기, 제외할 컬럼은 뺌
#     #numeric_cols = df.select_dtypes(include='number').columns
#     #target_cols = [col for col in numeric_cols if col not in exclude]
#
#     # 전체 컬럼 중에서 제외할 컬럼을 빼기
#     exclude_cols = [col for col in df.columns if col not in exclude]
#     for col in exclude_cols:
#         # 해당 컬럼에서 결측치 제외
#         series = df[col].dropna()
#         # 사분위수 (Q1, Q3) 계산
#         Q1 = series.quantile(0.25) # 데이터의 하위 25%
#         Q3 = series.quantile(0.75) # 데이터의 상위 25%
#         # Interquartile Range 계산
#         IQR = Q3 - Q1 # 데이터 중간 50%가 차지하는 범위
#
#         # 이상치 경계 계산
#         low_border = Q1 - 1.5 * IQR # Q1아래로 1.5배 더 떨어진 값
#         up_border = Q3 + 1.5 * IQR # Q3위로 1.5배 더 커진 값
#
#         # 이상치 탐지 - low_num 보다 작거나 , up_num 보다 크면 이상치
#         total_count = series.count() # 전체 유효 값 개수
#         low_count = (series < low_border).sum() # 하한 미만 값 개수
#         up_count = (series > up_border).sum() # 상한 초과 값 개수
#         outlier_count = low_count + up_count # 총 이상치 수
#         outlier_ratio = round((outlier_count / total_count) * 100, 2) # 이상치 비율 (%)
#
#         # 요약 리스트에 추가
#         summary_list.append({
#             '컬럼명': col,
#             '이상치 비율 (%)': outlier_ratio,
#             'Q1 (25%)': round(Q1, 2),
#             'Q3 (75%)': round(Q3, 2),
#             'IQR': round(IQR, 2),
#             '이상치 하한': round(low_border, 2),
#             '이상치 상한': round(up_border, 2),
#             '하한 미만 개수': low_count,
#             '상한 초과 개수': up_count,
#             '이상치 총 개수': outlier_count
#         })
#
#     return pd.DataFrame(summary_list)
#
# # 제외할 컬럼 지정 - 불필요한 컬럼과 위에서 결측치 처리한 컬럼
# # 결측치 처리전 원본 컬럼도 일단 뻬고
# exclude_cols = ['id', 'churn', 'fill_reamining_contract',
#                 'fill_download_avg', 'fill_upload_avg',
#                 'reamining_contract', 'download_avg',
#                 'upload_avg']
#
# # 함수 호출
# result = get_outlier_summary(df, exclude=exclude_cols)
#
# # 결과 확인
# result_sorted = result.sort_values(by='이상치 비율 (%)', ascending=False)
# display(result_sorted)

# print(df['is_tv_subscriber'].value_counts())
# round(df['is_tv_subscriber'].value_counts(normalize=True), 2)

# 컬럼값이 이진형(0과 1)인 컬럼만 보기 - 이상치 없으니까
binary_cols = [col for col in df.columns if set(df[col].dropna().unique()) <= {0, 1}]
print(binary_cols)

['is_tv_subscriber', 'is_movie_package_subscriber', 'churn']
